In [ ]:
# ============================
# STEP 1: MOUNT GOOGLE DRIVE
# ============================
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


# Change directory to your project
PROJECT_PATH = '/content/drive/Othercomputers/Laptop/0a.Nathan Model'
%cd "$PROJECT_PATH"

# ============================
# STEP 2: INSTALL DEPENDENCIES
# ============================
# Install requirements if you have a requirements.txt file
!pip install -qr "colab_requirements.txt"
#!pip install -q pyarrow==14.0.2 transformers[torch] datasets evaluate

# Check if torch is using GPU
import torch
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# ============================
# STEP 3: RUN YOUR NLP CODE
# ============================
print("Running model...")
!python start.py
#testing basic og function, tf rels = False => 87,71,74 full
#then add neg sampling before binary filter, tf rels = False => 87,70,72 3000   (74.35, 57.78, 65.03)
#span neg sampling and add rel neg sampling, tf rels = False => 88, 69, 71 => rel pr is low => I think if you neg sample the rels, you must teacher force the rels also
#span neg sampling, rel neg sampling, add prune rels after rel neg sampling to save space, tf rels = False  => this crashes, need to fix the rel rep code
#span neg sampling, rel neg sampling, tf rels = True   88,71,73   3000

#from what I see neg sampling ont eh spans only works with no tf on teh rels and no neg sampling on the rels

# BE tagger => span neg sampling => span filter with tagger scores => rel gen   86,68,70 3000  => uses less memory
# span neg sampling => BE tagger => span filter with tagger scores => rel gen   86,66,67 3000
# BE tagger => span neg sampling => span filter with tths scores => rel-neg-sampling => rel gen  87,70,72 3000
# BECO tagger => span neg sampling => span filter with tagger scores => rel-neg-sampling => rel gen 87,69,69 3000
# BE tagger => span neg sampling => span filter with tths scores => rel-neg-sampling => tf rels => rel gen  86,70,70 3000 => takes longer to train but will get there...
# BE tagger => span filter with tths scores => rel gen

# BE tagger => span filter with tths scores => span marker => rel gen   => 85,70,70 => so marking doesn't really help at all, go figure....


#approximating unicausla pair classification
#so to simulate unicausal we have to use only the end classifier losses and TF for eval, for real conditions, we need all the losses and no TF for eval and th etraining limits to arodun 40%
#it like neg sampling through, seems to be beneficial to model training in all scenarios, delays overfitting and gives better control of P/R

#separating berts for span and rel may help a bit, but do not really change the fundamental issues




Mounted at /content/drive
/content/drive/Othercomputers/Laptop/0a.Nathan Model
GPU Available: True
GPU: Tesla T4
Running model...
Start...
Loading and Preparing Data
The file /content/drive/Othercomputers/Laptop/0a.Nathan Model/data/unicausal/model_data_altlex.json is encoded with UTF-8 (no BOM).

seq_len and span_width stats, use these to tune max_seq_len and max_span_widths....
-------------------------------
seq_len stats (max, mean, sd): 132, 30.81, 14.33
span_width stats (max, mean, sd): 116, 14.75, 11.03
-------------------------------
Making the Dataloaders
batch 20 from training data loader:
tokens: [['Buffy', 'Summers', 'has', 'just', 'moved', 'to', 'Sunnydale', 'after', 'burning', 'down', 'her', 'old', 'school', "'s", 'gym', ',', 'and', 'hopes', 'to', 'escape', 'her', 'Slayer', 'duties', '.'], ['The', 'aim', 'of', 'cognition-oriented', 'treatments', ',', 'which', 'include', 'reality', 'orientation', 'and', 'cognitive', 'retraining', ',', 'is', 'the', 'reduction', 'of', 'cogni